In [16]:
import pandas as pd 
import pandas as np 
import matplotlib.pyplot

In [17]:
df=pd.read_csv("DatasetExos2.csv" , delimiter=";")
df.columns

Index(['ep (ms)', 'Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z',
       'ID', 'Label', 'Category', 'Set'],
      dtype='object')

1. drop the first 7 columns

In [18]:
df = df.drop(columns=df.columns[:7])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9009 entries, 0 to 9008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        9007 non-null   object 
 1   Label     9009 non-null   object 
 2   Category  9005 non-null   object 
 3   Set       9003 non-null   float64
dtypes: float64(1), object(3)
memory usage: 281.7+ KB


1.2 removing the null values 

In [19]:
df = df.dropna()
df

,ID,Label,Category,Set
0,B,bench,heavy,30.0
1,B,bench,heavy,30.0
2,B,bench,heavy,30.0
3,B,bench,heavy,30.0
4,B,bench,heavy,30.0
...,...,...,...,...
9004,E,row,medium,40.0
9005,E,row,medium,40.0
9006,E,row,medium,40.0
9007,E,row,medium,40.0


In [20]:
df = df[df["Set"]>0]
df["Set"].value_counts()

Set
51.0      200
23.0      200
19.0      200
1.0       175
55.0      174
         ... 
91.0       43
8.0        42
65.0       42
20.0       38
2000.0      1
Name: count, Length: 92, dtype: int64

In [21]:
Q1 = df["Set"].quantile(0.25)
Q3 = df["Set"].quantile(0.75)

intervals = [min(df["Set"]) , Q1 , Q3 , max(df["Set"])]
labels = ["Low" , "Mid" , "High"]
df["Set"] = pd.cut(df["Set"] , intervals , labels=labels)
df

C:\Users\abdo7\AppData\Local\Temp\ipykernel_2788\1302458868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Set"] = pd.cut(df["Set"] , intervals , labels=labels)


,ID,Label,Category,Set
0,B,bench,heavy,Mid
1,B,bench,heavy,Mid
2,B,bench,heavy,Mid
3,B,bench,heavy,Mid
4,B,bench,heavy,Mid
...,...,...,...,...
9004,E,row,medium,Mid
9005,E,row,medium,Mid
9006,E,row,medium,Mid
9007,E,row,medium,Mid


2. transaction are the rows for example : 
{B , bench , heavy , mid}
and the items are the values of each column in one item Set :

In [22]:
itemset =[]
for i in df.columns :
    for j in df[i].unique():
        itemset.append(j)
print(itemset)

print("item length : "+ str(len(itemset)))
print("transaction number : "+ str(len(df)))

['B', 'A', 'E', 'C', 'D', 'Z', 'bench', 'ohp', 'squat', 'dead', 'row', 'rest', 'raw', 'heavy', 'medium', 'sitting', 'standing', 'Mid', 'High', 'Low', nan]
item length : 21
transaction number : 8996


4. Build DatasetExos_2

In [23]:
df["Set"].value_counts()

Set
Mid     4411
Low     2245
High    2165
Name: count, dtype: int64

In [24]:

df.to_csv("ExoDataset_2.csv")

# PART 2 

In [25]:
def condidat_Gen(k , itemSet=None):
    if k==1:
        new_itemset = itemset
    else :
        new_itemset =[]
        for i in range(len(itemSet)-1):
            for j in range(i+1 , len(itemSet)): 
                if str(itemSet[i]) != "nan" and str(itemSet[j]) !="nan":
                    if isinstance(itemSet[i]  , list ): 
                        new_itemset.append(list(set(itemSet[i]) | set(itemSet[j])))
                    else :
                        new_itemset.append([itemSet[i],itemSet[j]])
    return new_itemset

In [26]:
def compute_Support(df , itemSet):
    condidat_support = {}
    # when we have a list of lists means that we're in diffrent gen and not the 1st one 
    if not isinstance(itemSet[0] , list) : 
        for i in itemSet:
            if i in df["ID"].unique():
                new_df = df[df["ID"]==i]

            elif i in df["Label"].unique():
                new_df = df[df["Label"]==i]

            elif i in df["Category"].unique():
                new_df = df[df["Category"]==i]

            elif i in df["Set"].unique():
                new_df = df[df["Set"]==i]
            condidat_support[i]=len(new_df)/len(df)
    else : 
        for j in itemSet:
            new_df = df.copy()
            for i in j : 
                if i in df["ID"].unique():
                    new_df = new_df[new_df["ID"]==i]

                elif i in df["Label"].unique():
                    new_df = new_df[new_df["Label"]==i]

                elif i in df["Category"].unique():
                    new_df = new_df[new_df["Category"]==i]

                elif i in df["Set"].unique():
                    new_df = new_df[new_df["Set"]==i]
            condidat_support[str(j)]=len(new_df)/len(df)
    return condidat_support

In [27]:
import ast
def remove_minSup(minSup , Support , itemset):
    for key in Support : 
        if Support[key] < minSup : 
            try : 
                itemset.remove(ast.literal_eval(key))
            except Exception as e:
                itemset.remove(key)
    return itemset

In [28]:
min_sup =0.13
itemCk = condidat_Gen(1)
SupportCk = compute_Support(df , itemCk)
Lk = remove_minSup(min_sup , SupportCk , itemCk)
print(SupportCk)

{'B': 0.09326367274344154, 'A': 0.33159181858603826, 'E': 0.29390840373499333, 'C': 0.16462872387727878, 'D': 0.11649622054246332, 'Z': 0.00011116051578479324, 'bench': 0.18508225878168075, 'ohp': 0.1857492218763895, 'squat': 0.17874610938194754, 'dead': 0.17018674966651845, 'row': 0.15673632725655848, 'rest': 0.1233881725211205, 'raw': 0.00011116051578479324, 'heavy': 0.3838372610048911, 'medium': 0.49277456647398843, 'sitting': 0.056691863050244554, 'standing': 0.06669630947087594, 'Mid': 0.490329035126723, 'High': 0.24066251667407737, 'Low': 0.24955535793686082, nan: 0.0}


In [29]:
result =Lk
while(len(Lk)>1):
    itemCk = condidat_Gen(2 , Lk)
    tempSupport = compute_Support(df , itemCk)
    SupportCk = SupportCk | tempSupport 
    Lk = remove_minSup(min_sup , tempSupport , itemCk)
    result +=Lk

result
SupportCk 

{'B': 0.09326367274344154,
 'A': 0.33159181858603826,
 'E': 0.29390840373499333,
 'C': 0.16462872387727878,
 'D': 0.11649622054246332,
 'Z': 0.00011116051578479324,
 'bench': 0.18508225878168075,
 'ohp': 0.1857492218763895,
 'squat': 0.17874610938194754,
 'dead': 0.17018674966651845,
 'row': 0.15673632725655848,
 'rest': 0.1233881725211205,
 'raw': 0.00011116051578479324,
 'heavy': 0.3838372610048911,
 'medium': 0.49277456647398843,
 'sitting': 0.056691863050244554,
 'standing': 0.06669630947087594,
 'Mid': 0.490329035126723,
 'High': 0.24066251667407737,
 'Low': 0.24955535793686082,
 nan: 0.0,
 "['A', 'E']": 0.0,
 "['A', 'C']": 0.0,
 "['A', 'bench']": 0.031013783903957315,
 "['A', 'ohp']": 0.0805913739439751,
 "['A', 'squat']": 0.06936416184971098,
 "['A', 'dead']": 0.05935971542907959,
 "['A', 'row']": 0.009004001778568253,
 "['A', 'heavy']": 0.12894619831036017,
 "['A', 'medium']": 0.12038683859493109,
 "['A', 'Mid']": 0.1526233881725211,
 "['A', 'High']": 0.07703423743886172,
 "['A

In [30]:
import itertools
def generate_all_possible_rules(frequent_itemsets):
    all_rules = []
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            for i in range(1, len(itemset)):
                for antecedent in itertools.combinations(itemset, i):
                    consequent = list(sorted(set(itemset) - set(antecedent)))
                    antecedent = list(sorted(antecedent))
                    all_rules.append((antecedent, consequent))
    return all_rules

unique_list = []
for item in result:
    if item not in unique_list and isinstance(item , list):
        unique_list.append(item)
all_rules = generate_all_possible_rules(unique_list)

for antecedent, consequent in all_rules:
    print(f"{antecedent} -> {consequent}")

['A'] -> ['Mid']
['Mid'] -> ['A']
['E'] -> ['medium']
['medium'] -> ['E']
['E'] -> ['Mid']
['Mid'] -> ['E']
['dead'] -> ['medium']
['medium'] -> ['dead']
['heavy'] -> ['Mid']
['Mid'] -> ['heavy']
['medium'] -> ['Mid']
['Mid'] -> ['medium']
['medium'] -> ['E', 'Mid']
['E'] -> ['Mid', 'medium']
['Mid'] -> ['E', 'medium']
['E', 'medium'] -> ['Mid']
['Mid', 'medium'] -> ['E']
['E', 'Mid'] -> ['medium']


In [ ]:
import ast 
def concatenate_and_remove_duplicates(list1, list2):
    seen = set()
    result = []
    for item in list1 + list2:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

Supports = []
for antecedent, consequent in all_rules:
    combined_list = concatenate_and_remove_duplicates(antecedent, consequent)
    for i in SupportCk :
        try : 
            if set(combined_list) == set(ast.literal_eval(i)):
                if len(antecedent)==1:
                    value = SupportCk[str(antecedent[0])]
                else :
                    value = SupportCk[str(antecedent)]
                Supports.append(SupportCk[i]/value)
                break
        except Exception as e:
            pass

for i in range(len(all_rules)): 
    try : 
        print(f"{all_rules[i]}       confidence : {Supports[i]}")
    except Exception as e : 
        pass
    

(['A'], ['Mid'])       confidence : 0.4602748910492792
(['Mid'], ['A'])       confidence : 0.31126728632963047
(['E'], ['medium'])       confidence : 0.5011346444780636
(['medium'], ['E'])       confidence : 0.29889465373336344
(['E'], ['Mid'])       confidence : 0.5427382753403933
(['Mid'], ['E'])       confidence : 0.3253230559963727
(['dead'], ['medium'])       confidence : 0.8941868060091444
(['medium'], ['dead'])       confidence : 0.3088202120460185
(['heavy'], ['Mid'])       confidence : 0.41673906747755574
(['Mid'], ['heavy'])       confidence : 0.32622987984583995
(['medium'], ['Mid'])       confidence : 0.5869614256711031
(['Mid'], ['medium'])       confidence : 0.5898889140784402
(['medium'], ['E', 'Mid'])       confidence : 0.2596435822242274
(['E'], ['Mid', 'medium'])       confidence : 0.43532526475037825
(['Mid'], ['E', 'medium'])       confidence : 0.2609385626841986
(['E', 'medium'], ['Mid'])       confidence : 0.8686792452830189
(['Mid', 'medium'], ['E'])       confid